still in progress

evaluate accuracy on different city

In [ ]:
!pip install patchify

In [ ]:
# Connecting to the Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
%matplotlib inline

In [ ]:
from PIL import Image
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
from osgeo import gdal

In [ ]:
# import dataset for Philadelphia
# Open the file:
image_new = gdal.Open('link to image chips from new city')
park_new = gdal.Open('link to park chips from new city')

# turn into array
image_array_new = image_new.ReadAsArray()
image_array_new = np.transpose(image_array_new, [1, 2, 0]) # transpose the bands to the third dimension
print(image_array_new.shape)

park_array_new = park_new.ReadAsArray()
park_array_new = np.expand_dims(park_array_new, axis = 2)
print(park_array_new.shape)

(2894, 2738, 5)
(2894, 2738, 1)


In [ ]:
# cropping satellite image

#have to be the same as saved patches
patch_size = 256
step = 48
scaler = MinMaxScaler()
image_dataset_new = []  

patches_img = patchify(image_array_new, (patch_size, patch_size, 5), step=step)  
for i in range(patches_img.shape[0]):
  for j in range(patches_img.shape[1]):
    single_patch_img = patches_img[i,j,:,:]             
    single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
    single_patch_img = single_patch_img[0]                              
    image_dataset_new.append(single_patch_img) 

image_dataset_new = np.array(image_dataset_new)
print("The shape for image dataset is:", image_dataset_new.shape)

# cropping park dataset
park_dataset_new = []
patches_img = patchify(park_array_new, (patch_size, patch_size, 1), step=step)  
for i in range(patches_img.shape[0]):
  for j in range(patches_img.shape[1]):
    single_patch_img = patches_img[i,j,:,:]
    single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
    single_patch_img = single_patch_img[0]                             
    park_dataset_new.append(single_patch_img)

park_dataset_new = np.array(park_dataset_new)
print("The shape for park dataset is:", park_dataset_new.shape)

The shape for image dataset is: (2860, 256, 256, 5)
The shape for park dataset is: (2860, 256, 256, 1)


In [ ]:
image_dataset_new[4][image_dataset_new[4] < 0] = 0

In [ ]:
from tensorflow.keras.optimizers import Adam
#from focal_loss import BinaryFocalLoss
import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.metrics import MeanIoU

In [ ]:

model_test = load_model('/content/drive/MyDrive/1Utrecht_University/THESIS/Seattle_data/model-unet-10_seattle_san_francisco.h5', compile = False)

model_test.compile(optimizer=Adam(learning_rate = 1e-4), loss='binary_crossentropy', metrics=['accuracy']) # use the same parameters as in original training!

In [ ]:
# get IoU for new dataset
y_pred = model_test.predict(image_dataset_new)
y_pred_thresholded = y_pred > 0.5
IOU_keras = MeanIoU(num_classes = 2)  
IOU_keras.update_state(y_pred_thresholded, park_dataset_new)
print("Mean IoU =", IOU_keras.result().numpy())

#for model trained on both Seattle and San Francisco - 0,42

Mean IoU = 0.42766362


In [ ]:
y_pred.shape

(2860, 256, 256, 1)

In [ ]:
y_pred[0][0][0]

array([0.5238025], dtype=float32)

In [ ]:
y_pred_thresholded[0][0][0]

array([ True])

In [ ]:
loss, acc = model_test.evaluate(image_dataset_new, park_dataset_new, verbose = 1)

90/90 [==============================] - 38s 282ms/step - loss: 1.2995 - accuracy: 0.7003


In [ ]:
loss
#1,2994

1.2994917631149292

In [ ]:
acc
#0,70

0.7002856731414795